In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, classification_report
from matplotlib import pyplot as plt

### 1 Load and prepare CEO data

In [ ]:
# read csv file
ceo = pd.read_csv('ceo-RCI_ERP_AD_COLLECTION_FINAL-sample-data-2022-09-28.csv', delimiter=',')
print(ceo.columns)

#print(len(ceo))
#ceo = ceo[ceo['OCS_2000'] == 'Terres forestieres']
#print(len(ceo))

# subset columns
ceo = ceo[['plotid', 'lon', 'lat', 'CHG_2000_2015']]

# add CNC column for classification
ceo['cnc'] = ceo['CHG_2000_2015'].apply(lambda x: 0 if x == 'Stable' else 1)
ceo.head(10)

### 2 Load and Prepare TS data

In [ ]:
# load data
#ts = pd.read_pickle('results_Landsat_ndfi_1995-01-01_2000-01-01_2015-12-31_0.25.pickle')
ts = pd.read_json('ts.json')
# see all columns
ts.columns

In [ ]:
# turn nan to 0
ts['gfc_lossyear'] = np.nan_to_num(ts['gfc_lossyear'])
# create a binary loss
ts['gfc_loss_binary'] = ts['gfc_lossyear'].apply(lambda x: 0 if x == 0 or x > 15 else 1)

In [ ]:
# create a binary deforestation
ts['tmf_def_binary'] = ts['tmf_defyear'].apply(lambda x: 0 if x > 1999 and x < 2016 else 1)

# create a binary degradation
ts['tmf_deg_binary'] = ts['tmf_degyear'].apply(lambda x: 0 if x > 1999 and x < 2016 else 1)

### 2.1 Select columns for classification

In [ ]:
cols_to_classify = [
    #'images', 'mon_images',
    'gfc_tc00', 
    'gfc_loss_binary',
    'tmf_2000', 
    'tmf_def_binary', 'tmf_deg_binary',
    'bfast_magnitude',
    'ccdc_magnitude', 
    'ltr_magnitude', 'ltr_dur', 'ltr_rate',
    'cusum_confidence', 'cusum_magnitude', 
    'ts_mean', 'ts_sd', # we add this anyway in the next cell
    'ts_min', 'ts_max',
    'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min'
]


### 2.2 Extract time-series statistics for each band and add to predicitive data columns

In [ ]:
bands = list(ts['ts'][1].keys())
print(' Bands available in dataset time-series: ')
print(bands) 
for band in bands:
    
    # add mean and SD value of time-series for each band
    ts[band + '_mean'] = ts['ts'].apply(lambda x: np.nanmean(np.array(x[band])))
    ts[band + '_sd'] = ts['ts'].apply(lambda x: np.nanstd(np.array(x[band])))
    
    # append to classification bands
    cols_to_classify.append(band + '_mean')
    cols_to_classify.append(band + '_sd')

In [ ]:
cols_to_classify

In [ ]:
print(' Number of predictive features: ' + str(len(cols_to_classify)))

### 2.3 Add a Kmeans column (because we can)

In [ ]:
nr_of_cluster=50

# run kmeans
kmeans_model = KMeans(n_clusters=nr_of_cluster, random_state=42).fit(ts[cols_to_classify])
ts['kmeans'] = kmeans_model.predict(ts[cols_to_classify])

cols_to_classify.append('kmeans')

### 3 Merge CEO and TS data

In [ ]:
df_class = pd.merge(ceo, ts[cols_to_classify + ['PLOTID']], how='inner', left_on='plotid', right_on='PLOTID')

In [ ]:
h.plot_stats_per_class(df_class, 'CHG_2000_2015', cols_to_classify)

### 4 Classification - Ensemble model creation

### 4.1 Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_class[cols_to_classify], df_class['cnc'], test_size=0.2, random_state=42, stratify=df_class['cnc'])

### 4.2 RF classifcation with automated model optimization for avoiding false positives in the stable class 

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, oob_score=True)

param_grid = {
    'min_samples_split': [10], 
    'n_estimators' : [500],
    #'max_depth': [3, 5, 15, 25],
    'max_features': [10]
}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}


def grid_search_wrapper(refit_score='recall_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    
    skf = StratifiedKFold(n_splits=5)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score, cv=skf, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train[cols_to_classify], y_train)

    # make the predictions
    y_pred = grid_search.predict(X_test[cols_to_classify])

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

grid_search_clf = grid_search_wrapper(refit_score='precision_score')

### 4.3 Model accuracy

In [ ]:
X_train['class'] = grid_search_clf.predict(X_train[cols_to_classify])
X_test['class'] = grid_search_clf.predict(X_test[cols_to_classify])

print('-----------------------------')
print(' Stats on train:')
print('-----------------------------')
print('Accuracy on train: ' + str(accuracy_score(y_train, X_train['class'])))
display(confusion_matrix(y_train, X_train['class']))

print('')
print('')
print('-----------------------------')
print(' Stats on test:')
print('-----------------------------')
#print('Out of Bag Error RF: ' + str(grid_search_clf.oob_score_))
print('Accuracy on test: ' + str(accuracy_score(y_test, X_test['class'], )))
cm = confusion_matrix(y_test, X_test['class'])
display(cm)

print('')
print('')
print('-----------------------------')
print(' Per class stats on test:')
print('-----------------------------')
print(classification_report(y_test, X_test['class'], target_names=['stable', 'change'], digits=4))

df_cm = pd.DataFrame(cm, index=['stable', 'change'], columns=['stable', 'change'])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

### 4.4 Feature Importance

In [ ]:
forest = grid_search_clf.best_estimator_
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
feature_names = [f"{i}" for i in X_train[cols_to_classify].columns]
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots(figsize=(20,10))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using RF")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

### 4.4 get threshhold for not having any change

In [ ]:
df_class[['prob_stable', 'prob_change']] = grid_search_clf.predict_proba(df_class[cols_to_classify][cols_to_classify])

print('Maximum probability of stable for change points')
thshld = df_class['prob_stable'][df_class['cnc'] == 1].max()
thshld = np.percentile(df_class['prob_stable'][df_class['cnc'] == 1], 90)
print(thshld)

print('Number of points above thshld (considered stable)')
abv_thshld = df_class[df_class['prob_stable'] > thshld]
print(str(len(abv_thshld)) + ' out of ' + str(len(df_class)))

print('Number of actual change points left in stable')
len(abv_thshld[abv_thshld['cnc'] == 1])

print(' Point Ids of interpreted change with a high probability of being stable')
display(df_class[(df_class['cnc'] == 1) & (df_class['prob_stable'] > 0.5)].sort_values('prob_stable', ascending=False))

### 5 Apply ensemble model to ALL points

In [ ]:
ts['cnc_class'] = grid_search_clf.predict(ts[cols_to_classify])
ts[['prob_stable', 'prob_change']] = grid_search_clf.predict_proba(ts[cols_to_classify])

print(' Number of points to visually recheck based on classification result (0.5 probability)')
print(len(ts[ts['cnc_class'] == 1]))


print(' Number of points to visually recheck based on adjusted probability')
print(len(ts[ts['prob_stable'] < thshld]))

### 6 Select most likely change points based on probability

In [ ]:
# Number of points you can afford to analyse
nr_of_points = 5000

# select
selection = ts[['PLOTID', 'prob_change']].sort_values('prob_change', ascending=False).head(nr_of_points)
display(selection)
print('Minimum change probablity included in selection: ' + str(selection['prob_change'].min()))

In [ ]:
print(' Number of points to visually recheck based on margin of classifier') 
ts[['PLOTID', 'prob_change']][(ts['prob_change'] > 0.45) & (ts['prob_change'] < 0.55)]